
# MCP Management with LlamaStack

This notebook demonstrates how to manage Model Context Protocol (MCP) toolgroups using the `llama-stack-client`. It offers a hands-on introduction to how MCP allows tools to be dynamically registered and invoked within a llama-stack based system. 

###  What is MCP?

Model Context Protocol (MCP) is a mechanism that allows agents to interact with tools from a registered endpoint. These MCP tools are used to extend the capabilities of an agent in real time.

To use MCP tools, you need:
- A running MCP server with a valid endpoint URL.
- An application hosting the MCP client.

## Prerequisites 

Before starting, ensure:

- You have a running instance of a LlamaStack server ([local](../../../../local_setup_guide.md) or [remote](../../../../kubernetes/)).
- You have configured with the following variables:
  
```
REMOTE_BASE_URL=http://your-llamastack-url
MCP_SERVER_URL=http://your-mcp-url
```

In [1]:
import os
from llama_stack_client import LlamaStackClient
from dotenv import load_dotenv
load_dotenv()

base_url=os.getenv("REMOTE_BASE_URL")
# connecting to remote server
client = LlamaStackClient(base_url=base_url)

## View Registered tool groups

Refer to this [kubernetes/mcp-servers/](../../../../kubernetes/mcp-servers) to see how to deploy a number of MCP servers on OpenShift.

To inspect which toolgroups are currently registered with your LlamaStack server, use:

```
client.toolgroups.list()
```

In [2]:
registered_toolgroups = {tg.identifier for tg in client.toolgroups.list()}

print("Registered Toolgroups:")
for tg in registered_toolgroups:
    print(f" - {tg}")

Registered Toolgroups:
 - mcp::openshift
 - builtin::rag
 - mcp::ansible
 - mcp::slack
 - builtin::code_interpreter
 - builtin::websearch
 - mcp::github


## Register an MCP tool group 

Next, we register a new MCP tool group using `client.toolgroups.register(...)`.

Ensure that the provided MCP endpoint is valid and accessible.

In [3]:
# Register MCP tools
mcp_custom_url = os.getenv("CUSTOM_MCP_SERVER_URL")
if "mcp::custom_tool" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="mcp::custom_tool",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":mcp_custom_url}
    )

### Verify Registered tools
After registering a new MCP tool group, verify its presence by listing all currently registered tool groups using:

```
client.toolgroups.list()
``` 

In [4]:
registered_toolgroups = {tg.identifier for tg in client.toolgroups.list()}

print("Registered Toolgroups:")
for tg in registered_toolgroups:
    print(f" - {tg}")

Registered Toolgroups:
 - mcp::openshift
 - builtin::rag
 - mcp::custom_tool
 - mcp::ansible
 - mcp::slack
 - builtin::code_interpreter
 - builtin::websearch
 - mcp::github


You should now see the custom tool group included in the list of registered tool groups.

##  View Tools in a Tool group

Let us inspect the tools available within a specific tool group, such as `mcp::custom_tool`.

In [5]:
tools = client.tools.list(toolgroup_id="mcp::custom_tool")
tool_names = [t.identifier for t in tools]

print(tool_names)

['generate_random_number', 'approve_score']


##  Unregistering MCP Tool groups

If a tool group (e.g., `mcp::custom_tool`) is already registered, you can unregister it to prevent conflicts when re-registering or updating it.

In [6]:
client.toolgroups.unregister(toolgroup_id="mcp::custom_tool") 

To check whether a specific tool group such as `mcp::custom_tool` is currently registered, use the listing command:

```
client.toolgroups.list()
``` 


In [7]:
registered_toolgroups = {tg.identifier for tg in client.toolgroups.list()}

print("Registered Toolgroups:")
for tg in registered_toolgroups:
    print(f" - {tg}")

Registered Toolgroups:
 - mcp::openshift
 - builtin::rag
 - mcp::ansible
 - mcp::slack
 - builtin::code_interpreter
 - builtin::websearch
 - mcp::github


As expected, you will no longer see `custom_tool` listed in the registered tool groups after it has been unregistered.

## Summary

In this notebook, we walked through the process of managing Model Context Protocol (MCP) tool groups within the LlamaStack ecosystem. Here is a step-by-step summary of what we accomplished:

- **Connected to a LlamaStack server** using environment variables (e.g., `REMOTE_BASE_URL`) to initialize the `llama-stack-client`.

- **Viewed registered tool groups** to understand the currently available MCP integrations.

- **Queried tools in a specific group**, such as `mcp::custom_tool`, to explore the capabilities exposed by an MCP server.

- **Unregistered tool groups** when needed—for instance, before re-registering with an updated endpoint.

- **Registered a new MCP tool group** using a `toolgroup_id`, specifying the provider type (`model-context-protocol`), and supplying the MCP server URL (`CUSTOM_MCP_SERVER_URL`).

- **Verified registration** by checking that the tool group appeared in the active tool listing and its tools were accessible to agents.